In [1]:
!pip install pandas openai pymongo pypdf langchain_community tqdm tenacity scikit-learn tiktoken python-dotenv seaborn --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.1/669.1 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

# RAG System with MongoDB as vector search engine

## Parsing the pdf and cleaning it

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# The url for the user manual pdf.
url = "https://raw.githubusercontent.com/MatthewHsu1/RAGsystemOpenAI/main/F80_F83_F85_OM_810.pdf"

# Initialize PyPDFLoader with downloaded user manual pdf
loader = PyPDFLoader(url)
pages = loader.load_and_split()

In [ ]:
# First create a dictionary of headers to parse the pdf by headers.

headers = [
    "TABLE OF CONTENTS",
    "PRODUCT REGISTRATION",
    "IMPORTANT SAFETY INSTRUCTIONS",
    "IMPORTANT ELECTRICAL INSTRUCTIONS",
    "GROUNDING INSTRUCTIONS",
    "IMPORTANT OPERATION INSTRUCTIONS",
    "IMPORTANT SAFETY INSTRUCTIONS",
    "IMPORTANT SAFETY INSTRUCTIONS",
    "PREVENTATIVE MAINTENANCE CHART",
    "F80 / F83 / F83 ASSEMBLY PACK CHECKLIST",
    "F80 / F83 / F83 ASSEMBLY INSTRUCTIONS",
    "FOLDING INSTRUCTIONS",
    "TRANSPORTATION INSTRUCTIONS",
    "OPERATION OF YOUR TREADMILL",
    "GETTING STARTED",
    "QUICK-START OPERATION",
    "PAUSE/STOP/RESET FEATURE",
    "INCLINE FEATURE",
    "DOT MATRIX CENTER DISPLAY",
    "PROGRAMMABLE FEATURES",
    "HEART RATE PROGRAMS",
    "USING HEART RATE TRANSMITTER",
    "GENERAL MAINTENANCE",
    "BELT ADJUSTMENTS",
    "TREAD-BELT TRACKING ADJUSTMENT",
    "BELT/DECK LUBRICATION",
    "SERVICE CHECKLIST - DIAGNOSIS GUIDE",
    "MANUFACTURER’S LIMITED WARRANTY"
]

In [ ]:
import re

def strip_top(page):
  """
    Every document inside the variable of pages has this string in the beginning, 'F80 /  F83 / F85 TREADMILL'.
    Therefore, the real content of the pdf begins after that string.
  """
  pattern = r"F80 /  F83 / F85 TREADMILL(.*)"
  for page in pages:
    match = re.search(pattern, page.page_content, re.DOTALL)

    clean_text = ""
    if match:
        clean_text = match.group(1).strip()
        clean_text = clean_text.replace("\n", " ")
        clean_text = clean_text.replace("\x84", "")
        clean_text = re.sub(r'\s+', ' ', clean_text)
        page.page_content = clean_text
    else:
      clean_text = page.page_content.replace("\n", " ")
      clean_text = clean_text.replace("\x84", "")
      clean_text = re.sub(r'\s+', ' ', clean_text)
      page.page_content = clean_text

In [ ]:
# The pdf is in plain text
strip_top(pages)

In [ ]:
import re

def split_by_header(headers, pages):

  list_of_string = []
  for page in pages:

    # Create a regular expression that pattern matches all strings in headers.
    pattern = '|'.join(map(re.escape, headers))

    # Split each page by the pattern created.
    split_result = re.split(pattern, page.page_content)

    # Get rid of any empty strings
    split_result = [s.strip() for s in split_result if s.strip() != '']

    list_of_string += split_result

  return list_of_string

In [ ]:
text = split_by_header(headers, pages)

In [ ]:
# Check to see if any string in the list has the possibility to exceed the text embedding token limit.

for string in text:
  word_count = len(string.split())
  assert (word_count <= 5500) == True, "A string may potentially exceed the token limit"

In [ ]:
#Convert the list 'text' into a panda dataframe.

import pandas as pd

dataset_df = pd.DataFrame(text)
dataset_df.columns = ['text']

(43, 1)

## Transform the pdf into vector embeddings and store it inside of MongoDB

In [ ]:
import openai
from google.colab import userdata

openai.api_key = userdata.get('open_ai')

EMBEDDING_MODEL = 'text-embedding-3-small'

# This method will take a text and get an vector embedding from OpenAI
def get_embedding(text):
  if not text or not isinstance(text, str):
    return None

  try:
    embedding = openai.embeddings.create(input=text, model=EMBEDDING_MODEL).data[0].embedding
    return embedding
  except Exception as e:
    print(f'Error in get_embedding: {e}')
    return None

dataset_df['embedding'] = dataset_df['text'].apply(get_embedding)

dataset_df.head()

In [ ]:
import pymongo
from google.colab import userdata

# This method will connect to mongoDB and get a client instance.
def get_mongo_client(mongo_uri):
  try:
    client = pymongo.MongoClient(mongo_uri)
    print('Connection to MongoDB successful')
    return client
  except pymongo.errors.ConnectionFailure as e:
    print(f'Connection failed: {e}')
    return None

mongo_uri = userdata.get('mongo_url')
if not mongo_uri:
  print('Mongo_uri not set in envirnment variables')

mongo_client = get_mongo_client(mongo_uri)

db = mongo_client['Treadmill']
collection = db['F80_F83_F85']

documents = dataset_df.to_dict('records')
collection.insert_many(documents)

print('Data ingestion into MongoDB completed')

Connection to MongoDB successful


## Below is the query part

In [ ]:
import openai
from google.colab import userdata

openai.api_key = userdata.get('open_ai')

EMBEDDING_MODEL = 'text-embedding-3-small'

# This method will take a text and get an vector embedding from OpenAI
def get_embedding(text):
  if not text or not isinstance(text, str):
    return None

  try:
    embedding = openai.embeddings.create(input=text, model=EMBEDDING_MODEL).data[0].embedding
    return embedding
  except Exception as e:
    print(f'Error in get_embedding: {e}')
    return None

In [ ]:
import pymongo
from google.colab import userdata

# This method will connect to mongoDB and get a client instance.
def get_mongo_client(mongo_uri):
  try:
    client = pymongo.MongoClient(mongo_uri)
    print('Connection to MongoDB successful')
    return client
  except pymongo.errors.ConnectionFailure as e:
    print(f'Connection failed: {e}')
    return None

mongo_uri = userdata.get('mongo_url')
if not mongo_uri:
  print('Mongo_uri not set in envirnment variables')

mongo_client = get_mongo_client(mongo_uri)

db = mongo_client['Treadmill']
collection = db['F80_F83_F85']

Connection to MongoDB successful


In [ ]:
def vector_search(user_query, collection):
  query_embedding = get_embedding(user_query)

  if query_embedding is None:
    return "Invalid query or embedding generation failed."

  pipeline = [
      {
          "$vectorSearch": {
              "index": "vector_index",
              "queryVector": query_embedding,
              "path": "embedding",
              "numCandidates": 10,
              "limit": 10
          }
      },
      {
          "$project": {
              "text":1,
              "score": {
                  "$meta": "vectorSearchScore"
              }
          }
      }
  ]

  results = collection.aggregate(pipeline)
  return list(results)

In [ ]:
def handle_user_query(query, collection):
  get_knowledge = vector_search(query, collection)

  search_result = ''
  for result in get_knowledge:
    search_result += f"Instruction: {result.get('text', 'N/A')} \n"

  completion = openai.chat.completions.create(
      model='gpt-3.5-turbo',
      messages=[
          {'role': "system", 'content': 'You are a treadmill technician that provides helpful information by forming a cohisive answer with the context given.'},
          {'role': 'user', 'content':
            'Context: ' + search_result + '\n' +
            'question: ' + query + '\n' +
            'State each step and then show your work for performing that step.' + '\n' +
            'steps: - what facts do you need to look for in the text to answer the question?' +
                    '- Does the text contain all the facts needed to answer the question?' +
                    '- Think about how you might answer the question given what you know.' +
                    '- If you don\'t have enough facts answer "<Reponse> Sorry I don\'t have enough information."' +
                    '- Otherwise, answer the question and prefix your answer with <response>.'
          }
      ]
  )

  return (completion.choices[0].message.content), search_result

## Result

In [ ]:
query = "Good weather ehh ?"
response, source_information = handle_user_query(query, collection)

print(f"Response: {response} \n")
print(f"Source Information: \n{source_information}")

Response: steps:
- Look for information in the text related to weather or climate conditions.
- Check for any mentions of outdoor workout recommendations or treadmill features related to weather.
- Consider how heart rate programs or other features could be affected by weather conditions.
- If the text does not provide information related to weather, mention the lack of relevant details.

<response> Sorry I don't have enough information. 

Source Information: 
Instruction: F80 CONSOLEGETTING FAMILIAR WITH THE CONTROL PANEL F83 / 85 CONSOLECOOLING FANS SPEAKER FAN POWER SWITCH HEADPHONE JACK CONTACT HEART RATE SENSORSSPEAKERINCLINE QUICK KEYS CONTROL KEYS AUDIO IN JACK (MP3, CD, OR SMARTPHONE)SPEED QUICK KEYS DISPLAY BUTTONDOT MATRIX DISPLAY COOLING FANSSPEAKER FAN POWER SWITCH HEADPHONE JACKSPEAKERINCLINE QUICK KEYS CONTROL KEYS AUDIO IN JACK (MP3, CD, OR SMARTPHONE)SPEED QUICK KEYS DISPLAY BUTTONDOT MATRIX DISPLAY CONTACT HEART RATE SENSORS 
Instruction: F80_F83_F85_20111222 
Instruct

# RAG system with Qdrant as vertor search engine and fine-tuning on the model.

In [2]:
import json
import os
import time

from google.colab import userdata
import pandas as pd
from openai import OpenAI
import tiktoken
import seaborn as sns
from tenacity import retry, wait_exponential
from tqdm import tqdm
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings('ignore')

tqdm.pandas()

client = OpenAI(api_key=userdata.get("open_ai"))

## Download data

In [ ]:
!mkdir -p local_cache
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -O local_cache/train.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json -O local_cache/dev.json

## Read JSON to DataFrame

In [28]:
def json_to_dataframe_with_titles(json_data):
  qas = []
  context = []
  is_impossible = []
  answers = []
  titles = []

  for article in json_data['data']:
    title = article['title']
    for paragraph in article['paragraphs']:
      for qa in paragraph['qas']:
        qas.append(qa['question'].strip())
        context.append(paragraph['context'])
        is_impossible.append(qa['is_impossible'])

        ans_list = []
        for ans in qa['answers']:
          ans_list.append(ans['text'])
        answers.append(ans_list)
        titles.append(title)

  df = pd.DataFrame({'title': titles, 'question': qas, 'context': context, 'is_impossible': is_impossible, 'answer': answers})
  return df

In [42]:
def get_diverse_sample(df, sample_size=100, random_state=42):
  """
  Get a diverse sample of the dataframe by sampling from each title
  """
  sample_df = df.groupby(['title', 'is_impossible']).apply(lambda x: x.sample(min(len(x), max(1, sample_size // 50)), random_state=random_state)).reset_index(drop=True)

  if len(sample_df) < sample_size:
    remaining_sample_size = sample_size - len(sample_df)
    remaining_df = df.drop(sample_df.index).sample(remaining_sample_size, random_state=random_state)
    sample_df = pd.concat([sample_df, remaining_df]).sample(frac=1, random_state=random_state).reset_index(drop=True)

  return sample_df.sample(min(sample_size, len(sample_df)), random_state=random_state).reset_index(drop=True)

In [38]:
train_df = json_to_dataframe_with_titles(json.load(open('local_cache/train.json')))
val_df = json_to_dataframe_with_titles(json.load(open('local_cache/dev.json')))

In [49]:
df = get_diverse_sample(val_df, sample_size=100, random_state=42)
df.head()

,title,question,context,is_impossible,answer
0,Scottish_Parliament,What consequence of establishing the Scottish ...,A procedural consequence of the establishment ...,False,[able to vote on domestic legislation that app...
1,Imperialism,Imperialism is less often associated with whic...,The principles of imperialism are often genera...,True,[]
2,Economic_inequality,What issues can't prevent women from working o...,"When a person’s capabilities are lowered, they...",True,[]
3,Southern_California,"What county are Los Angeles, Orange, San Diego...","Its counties of Los Angeles, Orange, San Diego...",True,[]
4,French_and_Indian_War,When was the deportation of Canadians?,Britain gained control of French Canada and Ac...,True,[]


## Answer using Base gpt-3.5-turbo model

Zero Shot Prompt

In [50]:
def get_prompt(row):
  return [
      {"role": "system", "content": "You are a helpful assistant."},
      {
          "role": "user",
          "content": f"""Answer the following Question based on the Context only. Only answer from the Context. If you don't know the answer, say 'I don't know'.
          Question: {row.question}\n\n
          Context: {row.context}\n\n
          Answer:\n"""
      }
  ]

Answering using Zero Shot Prompt

In [56]:
@retry(wait=wait_exponential(multiplier=1, min=2, max=6))
def api_call(messages, model):
  return client.chat.completions.create(
      model=model,
      messages=messages,
      stop=["\n\n"],
      max_tokens=100,
      temperature=0.0,
  )

def answer_question(row, prompt_func=get_prompt, model="gpt-3.5-turbo"):
  messages = prompt_func(row)
  response = api_call(messages, model)
  return response.choices[0].message.content

In [59]:
df["generated_answer"] = df.progress_apply(answer_question, axis=1)
df.to_json("local_cache/100_val.json", orient="records", lines=True)
df = pd.read_json("local_cache/100_val.json", orient="records", lines=True)

100%|██████████| 100/100 [01:18<00:00,  1.28it/s]


In [61]:
df.head()

,title,question,context,is_impossible,answer,generated_answer
0,Scottish_Parliament,What consequence of establishing the Scottish ...,A procedural consequence of the establishment ...,False,[able to vote on domestic legislation that app...,Scottish MPs sitting in the UK House of Common...
1,Imperialism,Imperialism is less often associated with whic...,The principles of imperialism are often genera...,True,[],I don't know.
2,Economic_inequality,What issues can't prevent women from working o...,"When a person’s capabilities are lowered, they...",True,[],Gender roles and customs.
3,Southern_California,"What county are Los Angeles, Orange, San Diego...","Its counties of Los Angeles, Orange, San Diego...",True,[],California
4,French_and_Indian_War,When was the deportation of Canadians?,Britain gained control of French Canada and Ac...,True,[],1755


## Fine-tuning and Answering using Fine-tuned modal